# Panel Data

Ya hemos visto cómo se predicen diferentes series temporales basadas en un único elemento de estudio, es decir, una serie temporal. Sin embargo, ¿cómo podemos hacer para predecir más de un elemento a la vez, es decir, cómo podemos predecir Panel Data?

Pues la solución pasa por utilizar algoritmos de Machine Learning. Podemos utilizar estos algoritmos para detectar los patrones en base a sus últimos datos, aunque también ayudaría si tenemos datos característicos de cada individuo, de modo que le podamos proveer de más información al modelo.

Cabe destacar, que estos modelos se suelen utilizar para predecir el siguiente punto, no para hacer forecast de varios puntos.

A continuación, te reto a que intentes predecir valores de un panel data, en este caso, deberemos predecir ``quantity``. Para ello:
1. Lee el dataframe de los mercados
2. Modifica el DataFrame para quedarte con los mercados que tengan todos los datos desde enero de 2010 hasta diciembre de 2015


In [179]:
import pandas as pd
import numpy as np

In [180]:
#1
df = pd.read_csv("data/MarketArrivals.csv")
df

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city,date
0,ABOHAR(PB),January,2005,2350,404,493,446,PB,ABOHAR,January-2005
1,ABOHAR(PB),January,2006,900,487,638,563,PB,ABOHAR,January-2006
2,ABOHAR(PB),January,2010,790,1283,1592,1460,PB,ABOHAR,January-2010
3,ABOHAR(PB),January,2011,245,3067,3750,3433,PB,ABOHAR,January-2011
4,ABOHAR(PB),January,2012,1035,523,686,605,PB,ABOHAR,January-2012
...,...,...,...,...,...,...,...,...,...,...
10222,YEOLA(MS),December,2011,131326,282,612,526,MS,YEOLA,December-2011
10223,YEOLA(MS),December,2012,207066,485,1327,1136,MS,YEOLA,December-2012
10224,YEOLA(MS),December,2013,215883,472,1427,1177,MS,YEOLA,December-2013
10225,YEOLA(MS),December,2014,201077,446,1654,1456,MS,YEOLA,December-2014


In [1]:
#2


### Función rolling

Antes de proseguir, vamos a ver la función ``rolling`` para hacer ventanas temporales. Esta función cobra especial sentido en estos modelos, pues nos ayudará a darle más contexto al algoritmo para predecir. Básicamente, consiste en llevar una ventana de un tamaño determinado desde diferentes puntos temporales, aplicando filtros particulares para cada una.

Veamos cómo funciona para calcular el precio máximo de los 6 meses anteriores, y del precio mínimo los 3 meses anteriores:

In [184]:
# Ordenamos los datos en base al mercado y a la fecha:
df2 = df.copy()
df2 = df2.sort_values(by=['market', 'date'], ascending=True)
df2 = df2.reset_index(drop=True)
df2.tail()

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city,date
279,RAIPUR(CHGARH),August,2015,540,3950,4350,4275,HR,RAIPUR,2015-08-01
280,RAIPUR(CHGARH),September,2015,330,4600,5000,4900,HR,RAIPUR,2015-09-01
281,RAIPUR(CHGARH),October,2015,5608,2330,3470,3300,HR,RAIPUR,2015-10-01
282,RAIPUR(CHGARH),November,2015,6040,1908,2317,2235,HR,RAIPUR,2015-11-01
283,RAIPUR(CHGARH),December,2015,7150,1475,1700,1608,HR,RAIPUR,2015-12-01


In [185]:
# Precio máximo para 6 meses:
df2_pricemax_6 = df2.groupby("market")['priceMax'].rolling(6).max().reset_index()
df2_pricemax_6 = df2_pricemax_6.drop(['level_1', 'market'], axis=1)
df2_pricemax_6 = df2_pricemax_6.rename({'priceMax': 'price_max_6'}, axis=1)
df2_pricemax_6.head(10)

,price_max_6
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,1644.0
6,1527.0
7,961.0
8,1421.0
9,1736.0


In [186]:
# Precio mínimo para 3 meses:
df2_pricemin_3 = df2.groupby("market")['priceMin'].rolling(3).min().reset_index()
df2_pricemin_3 = df2_pricemin_3.drop(['level_1', 'market'], axis=1)
df2_pricemin_3 = df2_pricemin_3.rename({'priceMin': 'price_min_3'}, axis=1)
df2_pricemin_3.head(5)

,price_min_3
0,NaN
1,NaN
2,780.0
3,659.0
4,498.0


In [187]:
# Juntamos todos los df auxiliares que nos hemos creado:
df_aux = df2_pricemax_6.join(df2_pricemin_3)
df_aux

,price_max_6,price_min_3
0,NaN,NaN
1,NaN,NaN
2,NaN,780.0
3,NaN,659.0
4,NaN,498.0
...,...,...
279,4350.0,1467.0
280,5000.0,2171.0
281,5000.0,2330.0
282,5000.0,1908.0


In [188]:
# Tras crear nuestro df auxiliar, lo unimos a los datos originales y eliminamos aquellos con nulos:
df_new = df.join(df_aux).dropna()
df_new = df_new.reset_index(drop=True)
df_new

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city,date,price_max_6,price_min_3
0,AMRITSAR(PB),January,2015,29900,1041,1500,1285,PB,AMRITSAR,2015-01-01,1644.0,468.0
1,AMRITSAR(PB),February,2010,11200,1338,1527,1430,PB,AMRITSAR,2010-02-01,1527.0,432.0
2,AMRITSAR(PB),February,2011,14700,1000,1277,1154,PB,AMRITSAR,2011-02-01,961.0,432.0
3,AMRITSAR(PB),February,2012,4400,550,650,613,PB,AMRITSAR,2012-02-01,1421.0,432.0
4,AMRITSAR(PB),February,2013,13100,1035,1527,1343,PB,AMRITSAR,2013-02-01,1736.0,684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
259,RAIPUR(CHGARH),December,2011,9315,702,847,775,HR,RAIPUR,2011-12-01,4350.0,1467.0
260,RAIPUR(CHGARH),December,2012,3850,1318,1618,1495,HR,RAIPUR,2012-12-01,5000.0,2171.0
261,RAIPUR(CHGARH),December,2013,5300,1644,1933,1804,HR,RAIPUR,2013-12-01,5000.0,2330.0
262,RAIPUR(CHGARH),December,2014,590,1400,1700,1600,HR,RAIPUR,2014-12-01,5000.0,1908.0


In [189]:
# Nos quedaríamos con las columnas que utilizaremos para predecir en nuestro dataset:
df2_end = df_new[['market', 'city', 'quantity', 'year', 'month', 'price_max_6', 'price_min_3']]
df2_end

,market,city,quantity,year,month,price_max_6,price_min_3
0,AMRITSAR(PB),AMRITSAR,29900,2015,January,1644.0,468.0
1,AMRITSAR(PB),AMRITSAR,11200,2010,February,1527.0,432.0
2,AMRITSAR(PB),AMRITSAR,14700,2011,February,961.0,432.0
3,AMRITSAR(PB),AMRITSAR,4400,2012,February,1421.0,432.0
4,AMRITSAR(PB),AMRITSAR,13100,2013,February,1736.0,684.0
...,...,...,...,...,...,...,...
259,RAIPUR(CHGARH),RAIPUR,9315,2011,December,4350.0,1467.0
260,RAIPUR(CHGARH),RAIPUR,3850,2012,December,5000.0,2171.0
261,RAIPUR(CHGARH),RAIPUR,5300,2013,December,5000.0,2330.0
262,RAIPUR(CHGARH),RAIPUR,590,2014,December,5000.0,1908.0


Ahora que ya conoces cómo utilizar esta función, aprovéchala para realizar los siguientes apartados:

Hemos reducido el DataFrame a los mercados que nos interesan, ahora nos queda adaptar los datos para predecir:
1. Construye un DataFrame con los diferentes lags para cada mercado. En algunos casos, cuando trabajamos con Panel Data, además de poner lags, se suelen poner agrupaciones de otras variables en base a diferentes ventanas temporales hacia atrás → función ``rolling`` de Pandas. Calcula el máximo del precio máximo de los últimos 3, 6 y 12 meses, así como la media de ``quantity`` en esas mismas ventanas temporales
2. Separa en train y test, basándote en el eje temporal, dejando el último mes para test
3. Prueba diferentes modelos de Machine Learning replicando la metodología vista para una serie temporal, pero con diferentes individuos, representando gráficamente los mejores resultados
  * Si utilizas un GridSearchCV, separa los CV en base a ``TimeSeriesSplit`` (de ``sklearn.model_selection``)